In [1]:
# Importation des packages 

import math
#import simpy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import scipy.stats as stats
from itertools import chain
from scipy.stats import norm
import matplotlib.pyplot as plt
import scipy.integrate as spi
from scipy.integrate import quad
from scipy.optimize import minimize
import statsmodels.api as sm


In [ ]:

data = pd.read_excel(r"C:\Users\Downloads\tb prod\data_TP2_A2023.xlsx")

data

,Strike,Ttm (days),Price
0,1150,14,44.7
1,1160,14,36.0
2,1170,14,28.0
3,1180,14,20.9
4,1190,14,14.9
...,...,...,...
85,1200,168,55.1
86,1210,168,49.8
87,1220,168,44.9
88,1230,168,40.4


Question 1

Question 2 

In [3]:
i = complex(0, 1)

def heston_model(S, K, r, T, kappa, lambda_, theta, sigma, rho, v):
    def f1(phi, S, K, r, T, kappa, lambda_, theta, sigma, rho, v):
        u1 = 0.5
        b1 = kappa + lambda_ - rho * sigma
        a = sigma**2

        d1 = np.sqrt(((rho*sigma*phi*i-b1)**2)-(2*u1*phi*i-phi**2)*a)
        g1 = (b1-rho*sigma*phi*i+d1)/(b1-rho*sigma*phi*i-d1)

        C1 = (r * i* phi* T) +((kappa*theta)/a)* ((b1 - rho * sigma * phi*i+ d1) * T - 2 * np.log((1 - g1*np.exp(d1 * T)) / (1 - g1)))
        D1 = ((b1 - rho * sigma * phi*i+ d1) / a)*((1 - np.exp(d1 * T)) / (1 - g1*np.exp(d1 * T)))

        f1 = np.exp(C1 + D1 * v + phi*i* np.log(S))
       
        return f1

    def f2(phi, S, K, r, T, kappa, lambda_, theta, sigma, rho, v):
        
        u2 = -0.5
        b2 = kappa + lambda_
        a = sigma**2
        d2 = np.sqrt(((rho*sigma*phi*i-b2)**2)-(2*u2*phi*i-phi**2)*a)
        g2 = (b2-rho*sigma*phi*i+d2)/(b2-rho*sigma*phi*i-d2)

        C2 = (r * i* phi* T) +((kappa*theta)/a)* ((b2 - rho * sigma * phi*i+ d2) * T - 2 * np.log((1 - g2*np.exp(d2 * T)) / (1 - g2)))
        D2 = ((b2 - rho * sigma * phi*i+ d2) / a)*((1 - np.exp(d2 * T)) / (1 - g2*np.exp(d2 * T)))

        f2 = np.exp(C2 + D2 * v + phi*i* np.log(S))

        return f2

    def integrand(fj, S, v, T, K, r, phi):
        return np.real(np.exp(-i * phi * np.log(K) + r * T) * fj(phi, S, K, r, T, kappa, lambda_, theta, sigma, rho, v) / (i * phi))

    integrand1 = lambda phi: integrand(f1, S, v, T, K, r, phi)
    integrand2 = lambda phi: integrand(f2, S, v, T, K, r, phi)

    P1 = 0.5 + (1 / np.pi) * spi.quad(integrand1, 0, np.inf)[0]
    P2 = 0.5 + (1 / np.pi) * spi.quad(integrand2, 0, np.inf)[0]

    return S * P1 - K * np.exp(-r * T) * P2

S = 1190.94
K = np.array(data["Strike"])
option_prices = np.array(data["Price"])
r = 0.0512
ech = np.array(data["Ttm (days)"])
days_to_expiry = ech / 365
v = 0.0212
kappa = 0.23
lambda_ = 0.5
theta = 0.037
sigma = 0.046
rho = -0.48
params_init = [kappa, lambda_, theta, sigma, rho, v]

def error_function(params):
    kappa, lambda_, theta, sigma, rho, v = params
    errors = []
    for j in range(len(K-1)):
        errors = np.sum(( option_prices[j] - heston_model(S, K[j], r, days_to_expiry[j],*params))**2)
    return np.sum(errors)


res = minimize(error_function,  params_init, method='Nelder-Mead')
calibrated_params = res.x   
# Affichage des résultats
print("Paramètres calibrés [kappa, lambda_, theta, sigma, rho, v] : ", calibrated_params)
print("Somme des erreurs au carré minimisée : ", res.fun  )
print("RMSE : ", np.sqrt(res.fun/90))


Paramètres calibrés [kappa, lambda_, theta, sigma, rho, v] :  [ 0.2303498   0.52038903  0.03651035  0.04763394 -0.48701413  0.02036414]
Somme des erreurs au carré minimisée :  4.748529566083464e-10
RMSE :  2.2969858428634166e-06


Question 3

Question 4

In [4]:
# Paramètres optimisés du modèle de Heston
v0 = 0.02036414  
theta = 0.03651035
sigma = 0.04763394  
rho = -0.48701413 
lambda_ = 0.52038903
r = 0.0512 

# Paramètres de l'option
S0 = 1190.94
T = 77/365 # temps restant jusqu'à l'échéance (en années)
K = 1200 # prix d'exercice
B = 1100 # niveau de la barrière
dt = 1/365 # pas de temps (en années)

n_paths = 100000

In [5]:
# Génération des trajectoires des variables stochastiques
np.random.seed(0)
Z1 = np.random.normal(0, 1, n_paths)* np.sqrt(dt) # innovation des rendements
Z2 = (rho * Z1 + np.sqrt(1 - rho**2) *(-Z1) )* np.sqrt(dt) # innovation de la variance

In [6]:
# Variables pour stocker les valeurs du call down&out pour chaque trajectoire
call_values = np.zeros(n_paths)

for i in range(n_paths):
    # Variables pour stocker les valeurs du sous-jacent et de la variance pour chaque période
    S = np.zeros(n_paths)
    v = np.zeros(n_paths)

    # Initialisation des valeurs initiales
    S[0] = S0
    v[0] = v0

    for j in range(1, n_paths): 
        # Calcul des valeurs du sous-jacent et de la variance
        v[j] = v[j-1] +(( kappa * (theta - v[j-1]) - lambda_* v[j-1]) * dt + sigma * np.sqrt(v[j-1]) * Z2[j])
        S[j] = S[j-1] * np.exp((r -0.5 * v[j-1]) * dt + np.sqrt(v[j-1]) * Z1[j])
       

        # Vérification de la barrière
        if S[j] <= B:
           S[j] = 0  # Si la barrière est touchée, la valeur du sous-jacent devient nulle

    # Calcul de la valeur du call européen down&out pour chaque trajectoire
    call_values[i] = np.exp(-r * T) * np.maximum(0, S[-1] - K)

# Calcul de la valeur estimée du call européen down&out
call_value = np.mean(call_values)

print("La valeur du call européen down&out est :", call_value)



Question 5

In [ ]:


payoff = np.zeros(n_paths)

for i in range(n_paths):
    S_values = np.zeros(n_paths)
    S_values[0] = S0

    for j in range(1, n_paths):
            
        S_values[j] = S_values[j-1] * np.exp((r - 0.5 * sigma**2) * dt + sigma  * Z1[j])

        if np.any(S_values < B):
            continue

        payoff[i] = np.exp(-r * T)*np.maximum(S_values[-1] - K, 0)
    
monte_carlo_estimate = np.mean(payoff)
    

print("Estimation de Monte Carlo de la valeur du call down&out :", monte_carlo_estimate)


Implied volatility using Newton Rapshon is:  0.144740098035505


Question 7

In [ ]:

data_bms = call_down_out_value
data_heston  = call_values
subdivisions = [40, 50, 100]

for subdivision in subdivisions:
    lots_heston = np.array_split(data_heston , subdivision)
    lots_bms = np.array_split(data_bms, subdivision)
    
    results_heston = []
    results_bms = []
    for k in range(subdivision):
        Y = lots_heston[k] 
        X = lots_bms[k] 
        X = sm.add_constant(X)
    # Effectuer la régression MCO
        model = sm.OLS(Y, X)
        results = model.fit()
        coefs = results.params  # coefficients estimés
        pvalues = results.pvalues  # valeurs-p
        r2 = results.rsquared 

# Afficher les résultats
    
        print(results.summary())
        print("Coefficients estimés:", coefs)
        print("Valeurs-p:", pvalues)
        print("R2 de la régression:", r2)
        print("------------------------------------------")

call_fin = np.mean(coefs[1]+ data_heston* coefs[2])
print("Prix du call avec variable de controle:" , call_fin)

